In [1]:
!pip install pyspark==2.4.5

     |████████████████████████████████| 217.8MB 36.7MB/s eta 0:00:01     |██████████████████████████▋     | 180.9MB 36.7MB/s eta 0:00:02     |████████████████████████████    | 190.7MB 36.7MB/s eta 0:00:01
     |████████████████████████████████| 204kB 36.5MB/s eta 0:00:01
  Stored in directory: /home/dsxuser/.cache/pip/wheels/bf/db/04/61d66a5939364e756eb1c1be4ec5bdce6e04047fc7929a3c3c
Successfully built pyspark


In [2]:
try:
    from pyspark import SparkContext, SparkConf
    from pyspark.sql import SparkSession
except ImportError as e:
    printmd('<<<<<!!!!! Please restart your kernel after installing Apache Spark !!!!!>>>>>')

In [3]:
spark = SparkSession.builder.getOrCreate()

In [18]:
# delete files from previous runs
!rm -f hmp.parquet*

# download the file containing the data in PARQUET format
!wget https://github.com/IBM/coursera/raw/master/hmp.parquet
    
# create a dataframe out of it
df = spark.read.parquet('hmp.parquet')

# register a corresponding query table
df.createOrReplaceTempView('df')

--2020-06-13 04:55:41--  https://github.com/IBM/coursera/raw/master/hmp.parquet
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/IBM/skillsnetwork/raw/master/hmp.parquet [following]
--2020-06-13 04:55:41--  https://github.com/IBM/skillsnetwork/raw/master/hmp.parquet
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/IBM/skillsnetwork/master/hmp.parquet [following]
--2020-06-13 04:55:42--  https://raw.githubusercontent.com/IBM/skillsnetwork/master/hmp.parquet
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.8.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.8.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 932997 (911K) [application/octet-stream]
Saving 

In [5]:
#Linear Regression

data1 = {'partno':100, 'maxtemp':35, "maxvibration":12, 'asperity':0.32}
data1 = {'partno':101, 'maxtemp':35, "maxvibration":21, 'asperity':0.32}
data1 = {'partno':130, 'maxtemp':46, "maxvibration":3412, 'asperity':12.42}
data1 = {'partno':131, 'maxtemp':48,"maxvibration":3512, 'asperity':13.43}

In [10]:
w1=0
w2=0
w3=0
w4=33

def linearPredict(data):
    return w1+w2*data["maxtemp"]+w3*data["maxvibration"]+w4*data["asperity"]

In [11]:
linearPredict(data1)

443.19

In [19]:
#Linear regression for HMP_Dataset

df.show(5)

+---+---+---+--------------------+-----------+
|  x|  y|  z|              source|      class|
+---+---+---+--------------------+-----------+
| 22| 49| 35|Accelerometer-201...|Brush_teeth|
| 22| 49| 35|Accelerometer-201...|Brush_teeth|
| 22| 52| 35|Accelerometer-201...|Brush_teeth|
| 22| 52| 35|Accelerometer-201...|Brush_teeth|
| 21| 52| 34|Accelerometer-201...|Brush_teeth|
+---+---+---+--------------------+-----------+
only showing top 5 rows



In [20]:
df = df.createOrReplaceTempView("df")
df_energy=spark.sql('''
select sqrt(sum(x*x)+sum(y*y)+sum(z*z)) as label,class from df group by class
''')

df_energy.createOrReplaceTempView("df_energy")
df_energy.show(5)

+------------------+-------------+
|             label|        class|
+------------------+-------------+
| 8959.680239829991|Use_telephone|
| 9737.511232342687|Standup_chair|
| 12542.96539897962|     Eat_meat|
|13225.945637269193|    Getup_bed|
|15003.269043778426|  Drink_glass|
+------------------+-------------+
only showing top 5 rows



In [22]:
#inner join to df for linear regression for x,y,z
df_join = spark.sql("""
select * from df inner join df_energy on df.class = df_energy.class 
""")

df_join.show(5)

+---+---+---+--------------------+-----------+-----------------+-----------+
|  x|  y|  z|              source|      class|            label|      class|
+---+---+---+--------------------+-----------+-----------------+-----------+
| 22| 49| 35|Accelerometer-201...|Brush_teeth|11785.39634462923|Brush_teeth|
| 22| 49| 35|Accelerometer-201...|Brush_teeth|11785.39634462923|Brush_teeth|
| 22| 52| 35|Accelerometer-201...|Brush_teeth|11785.39634462923|Brush_teeth|
| 22| 52| 35|Accelerometer-201...|Brush_teeth|11785.39634462923|Brush_teeth|
| 21| 52| 34|Accelerometer-201...|Brush_teeth|11785.39634462923|Brush_teeth|
+---+---+---+--------------------+-----------+-----------------+-----------+
only showing top 5 rows



In [27]:
#normalizing
from pyspark.ml.feature import VectorAssembler,Normalizer

va = VectorAssembler(inputCols=["x","y","z"],outputCol="features")

normalizer = Normalizer(inputCol="features",outputCol="features_norm",p=1.0)

# normalizer.transhow(5)

In [28]:
#Linear regression in pyspark
from pyspark.ml.regression import LinearRegression

linearRegression = LinearRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)


In [29]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[va,normalizer,linearRegression])

model = pipeline.fit(df_join)

predictions = model.transform(df_join)
predictions.show()

+---+---+---+--------------------+-----------+-----------------+-----------+----------------+--------------------+------------------+
|  x|  y|  z|              source|      class|            label|      class|        features|       features_norm|        prediction|
+---+---+---+--------------------+-----------+-----------------+-----------+----------------+--------------------+------------------+
| 22| 49| 35|Accelerometer-201...|Brush_teeth|11785.39634462923|Brush_teeth|[22.0,49.0,35.0]|[0.20754716981132...|12586.729735016828|
| 22| 49| 35|Accelerometer-201...|Brush_teeth|11785.39634462923|Brush_teeth|[22.0,49.0,35.0]|[0.20754716981132...|12586.729735016828|
| 22| 52| 35|Accelerometer-201...|Brush_teeth|11785.39634462923|Brush_teeth|[22.0,52.0,35.0]|[0.20183486238532...|12542.703337345756|
| 22| 52| 35|Accelerometer-201...|Brush_teeth|11785.39634462923|Brush_teeth|[22.0,52.0,35.0]|[0.20183486238532...|12542.703337345756|
| 21| 52| 34|Accelerometer-201...|Brush_teeth|11785.3963446292

In [34]:
#r2 measure of the model built
model.stages[2].summary.r2

0.03259100556263628

In [35]:
#Logistic Regression


In [45]:
# delete files from previous runs
!rm -f hmp.parquet*

# download the file containing the data in PARQUET format
!wget https://github.com/IBM/coursera/raw/master/hmp.parquet
    
# create a dataframe out of it
df = spark.read.parquet('hmp.parquet')

# register a corresponding query table
df.createOrReplaceTempView('df')

--2020-06-13 05:14:37--  https://github.com/IBM/coursera/raw/master/hmp.parquet
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/IBM/skillsnetwork/raw/master/hmp.parquet [following]
--2020-06-13 05:14:37--  https://github.com/IBM/skillsnetwork/raw/master/hmp.parquet
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/IBM/skillsnetwork/master/hmp.parquet [following]
--2020-06-13 05:14:37--  https://raw.githubusercontent.com/IBM/skillsnetwork/master/hmp.parquet
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.8.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.8.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 932997 (911K) [application/octet-stream]
Saving 

In [46]:
df.show(5)

+---+---+---+--------------------+-----------+
|  x|  y|  z|              source|      class|
+---+---+---+--------------------+-----------+
| 22| 49| 35|Accelerometer-201...|Brush_teeth|
| 22| 49| 35|Accelerometer-201...|Brush_teeth|
| 22| 52| 35|Accelerometer-201...|Brush_teeth|
| 22| 52| 35|Accelerometer-201...|Brush_teeth|
| 21| 52| 34|Accelerometer-201...|Brush_teeth|
+---+---+---+--------------------+-----------+
only showing top 5 rows



In [47]:
# df = df.createOrReplaceTempView("df")

In [48]:
splits = df.randomSplit([0.8, 0.2])
df_train = splits[0]
df_test = splits[1]

In [49]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import Normalizer


indexer = StringIndexer(inputCol="class", outputCol="label")

vectorAssembler = VectorAssembler(inputCols=["x","y","z"],
                                  outputCol="features")

normalizer = Normalizer(inputCol="features", outputCol="features_norm", p=1.0)

In [51]:
#logistic

from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)
pipeline = Pipeline(stages=[indexer, vectorAssembler, normalizer,lr])
model = pipeline.fit(df_train)
prediction = model.transform(df_test)
prediction.show(5)

+---+---+---+--------------------+-------------+-----+---------------+--------------------+--------------------+--------------------+----------+
|  x|  y|  z|              source|        class|label|       features|       features_norm|       rawPrediction|         probability|prediction|
+---+---+---+--------------------+-------------+-----+---------------+--------------------+--------------------+--------------------+----------+
|  0| 11| 38|Accelerometer-201...|Sitdown_chair|  8.0|[0.0,11.0,38.0]|[0.0,0.2244897959...|[1.25748310211891...|[0.20691496224784...|       0.0|
|  0| 16| 31|Accelerometer-201...|    Getup_bed|  1.0|[0.0,16.0,31.0]|[0.0,0.3404255319...|[1.25748310211891...|[0.20691496224784...|       0.0|
|  0| 26| 15|Accelerometer-201...| Climb_stairs|  4.0|[0.0,26.0,15.0]|[0.0,0.6341463414...|[1.25748310211891...|[0.20691496224784...|       0.0|
|  0| 27| 37|Accelerometer-201...|  Brush_teeth|  6.0|[0.0,27.0,37.0]|[0.0,0.421875,0.5...|[1.25748310211891...|[0.20691496224784.

In [52]:
prediction.printSchema()

root
 |-- x: integer (nullable = true)
 |-- y: integer (nullable = true)
 |-- z: integer (nullable = true)
 |-- source: string (nullable = true)
 |-- class: string (nullable = true)
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- features_norm: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [53]:
#performance evaluation

from pyspark.ml.evaluation import MulticlassClassificationEvaluator
MulticlassClassificationEvaluator().setMetricName("accuracy").evaluate(prediction) 

0.20535123804826652

Using random forest classifier for classification
https://spark.apache.org/docs/latest/ml-classification-regression.html#random-forest-classifier


In [54]:
df.show(5)

+---+---+---+--------------------+-----------+
|  x|  y|  z|              source|      class|
+---+---+---+--------------------+-----------+
| 22| 49| 35|Accelerometer-201...|Brush_teeth|
| 22| 49| 35|Accelerometer-201...|Brush_teeth|
| 22| 52| 35|Accelerometer-201...|Brush_teeth|
| 22| 52| 35|Accelerometer-201...|Brush_teeth|
| 21| 52| 34|Accelerometer-201...|Brush_teeth|
+---+---+---+--------------------+-----------+
only showing top 5 rows



In [66]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorIndexer

indexer = StringIndexer(inputCol="class", outputCol="label")

vectorAssembler = VectorAssembler(inputCols=["x","y","z"],
                                  outputCol="features")

# normalizer = Normalizer(inputCol="features", outputCol="features_norm", p=1.0)

featureIndexer =VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4)

rf = RandomForestClassifier(labelCol="label", featuresCol="indexedFeatures", numTrees=10)

pipeline = Pipeline(stages=[indexer, vectorAssembler, featureIndexer,rf])
model = pipeline.fit(df_train)
prediction = model.transform(df_test)
prediction.show(5)

+---+---+---+--------------------+-----------+-----+---------------+---------------+--------------------+--------------------+----------+
|  x|  y|  z|              source|      class|label|       features|indexedFeatures|       rawPrediction|         probability|prediction|
+---+---+---+--------------------+-----------+-----+---------------+---------------+--------------------+--------------------+----------+
|  0| 28| 28|Accelerometer-201...|Brush_teeth|  6.0|[0.0,28.0,28.0]|[0.0,28.0,28.0]|[3.10353327062450...|[0.31035332706245...|       0.0|
|  0| 28| 48|Accelerometer-201...|Brush_teeth|  6.0|[0.0,28.0,48.0]|[0.0,28.0,48.0]|[1.11450868665456...|[0.11145086866545...|       1.0|
|  0| 29| 34|Accelerometer-201...|       Walk|  0.0|[0.0,29.0,34.0]|[0.0,29.0,34.0]|[3.10353327062450...|[0.31035332706245...|       0.0|
|  0| 29| 39|Accelerometer-201...|Brush_teeth|  6.0|[0.0,29.0,39.0]|[0.0,29.0,39.0]|[3.10353327062450...|[0.31035332706245...|       0.0|
|  0| 30| 46|Accelerometer-201...|

In [72]:
predictions.select("prediction", "label", "features").show(5)


evaluator = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

rfModel = model.stages[2]
print(rfModel)  # summary only

+------------------+-----------------+----------------+
|        prediction|            label|        features|
+------------------+-----------------+----------------+
|12586.729735016828|11785.39634462923|[22.0,49.0,35.0]|
|12586.729735016828|11785.39634462923|[22.0,49.0,35.0]|
|12542.703337345756|11785.39634462923|[22.0,52.0,35.0]|
|12542.703337345756|11785.39634462923|[22.0,52.0,35.0]|
|12573.865911821156|11785.39634462923|[21.0,52.0,34.0]|
+------------------+-----------------+----------------+
only showing top 5 rows

Test Error = 1
VectorIndexer_39340e3a01f5


In [73]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
MulticlassClassificationEvaluator().setMetricName("accuracy").evaluate(prediction) 

0.44113668630453984